# Further Pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

df = pd.read_csv('data/countries.csv')

from IPython.display import HTML
np.random.seed(1)

## Pandas Basics

* Useful **methods** and **functions**
    * `describe`, `get_dummies`, etc.
* **Indexing**
    * `loc` vs. `iloc`
* **Column-wise** operations
    * `arithmetic`, `apply`, etc.
* **SQL** inspired functions
    * `merge`, `join`, `groupby`

In the Introduction to Pandas module we went over the basics of Pandas, covering DataFrames and the fundamental operations used to extract information from them.

We looked at useful methods and functions, multiple ways to index DataFrames, column-wise operations for applying functions to different attributes, and SQL-style functions. 

If you aren't familiar with any of these topics then it may be worth going back to recap.

## Advanced Processing Functions
* Pivot Tables
    * `pivot`, `pivot_table`, `melt`
* Multi-Index
    * `MultiIndex`, `stack`, `crosstab`
* Pandas `Series`  
    * `factorize`, `cut`

In this module we're going to talk about some of the advanced functionality `pandas` can offer us. 

The topics are broadly split up into three categories: pivot tables, multi-indices, and Series objects. 

These are three different advanced aspects of `pandas` that may appear less often than the fundamental operations but are nonetheless extremely useful to understand. You'll also find that their usage is fairly idiomatic of Python libraries in general, so a good understanding of the advanced `pandas` features is a very transferrable skillset!


## Pivot Tables
* Similar to pivot tables in **Excel**
* Reshape a dataset, so each row has a **category** and **subcategory** associated to a **value**
    * example: Great Britain's (category: country) online sales (subcategory: online) make a profit (value: profit) of X

Pivot tables in `pandas` are very similar to pivot tables in Excel. The idea behind them is to reshape a dataframe such that each row of the index column represents an individual category, with subcategories and values stored in separate columns.

For example, we could consider a dataset with profits from companies based in different countries split into subcategories: online and in-store sales. 

These subcategories have associated values (in this case profits).


## Example

In [ ]:
import pandas as pd
df = pd.read_csv('data/profits.csv')

df.head()

* The **category** is the **country**.
* The **sub-category** is the **type**.
* The **value** is the **profit**

We want to pivot the table so that there is only one copy of each country in the country column.

Consider this example where we have online and in-store sales for two countries. We have them stored in a flat format, so there are duplicates in the 'Country' and the 'Type' columns. 

Using pivot tables we can reshape these into a dataframe which has 'Country' as the broad category, 'Type' as the subcategory, and 'Profit' as the value.

The goal of the pivot table is to reshape the elements so that we have 'Country' as an index (i.e. there is one of each country in the country column).

## Example `df.pivot`:

In [ ]:
df.pivot(index='Country', columns='Type')

To use `df.pivot`, we provide a new index, which in this case is 'Country', which is the column we would like to compress down to one of each type. 

In order to do this we need to create new columns for each subcategory: here, the subcategories are in-store and online sales, which previously were defined in the column 'Type'. 

In this example we specify the index 'Country' and the column(s) 'Type' and pandas reshapes the dataframe into a format with 'Country' as the index, and columns specified by the possible values of 'Type'.

## Pivot Tables

There are two functions to build pivot tables in pandas:
* `.pivot(index, columns, [values])`
    * **numerical** and **categorical** data
    * no aggregation - will fail if there are **duplicate rows** with same categories
* `.pivot_table(index, columns, [values], [aggfunc])`
    * only with **numerical** data
    * can do aggregation (similar to `groupby`)

There are two different ways to make pivot tables in `pandas`. The first we have already seen: `.pivot`. 

`.pivot` takes two compulsory arguments (index and columns) but can also be specified to be applied only to values (profits here) that meet a certain criterion. `.pivot` can be applied to both numerical and categorical data but as a consequence won't do any aggregation. If, in the example above, there had been multiple rows for GB in-store sales, `.pivot` would have thrown an exception because it doesn't know how to combine them. 

By contrast, `pivot_table` only works with numerical data and so can aggregate duplicate rows. We can specify an aggregation function --- e.g. in the example above we may have wanted to *sum* the different sales subcategories over several quarters or years.

## `df.pivot`

* `.pivot` works with numerical and categorical data (if rows are unique per subcategory)
    * `index`: the column that will become the new index
    * `columns`: the subcategories to split by
    * `values`: [optional] if you want to keep subset of columns

To recap, `.pivot` works with **both** numerical **and** categorical data - with the requirement that rows are unique per subcategory. 

We must define the new index, as well as the subcategories we're going to split the data into. We can optionally specify a subset of the columns to keep using the keyword `values`. 

## `df.pivot`

In the previous example, what was the original index?

In [ ]:
df.head()

Is there another column that we could use as the index?

In the previous example, we specified that the new index was 'Country'. What was the original index? 

We hadn't supplied one, so pandas had automatically constructed an index from 0 to 3 for us. So when we remove the duplicates by splitting up the profits into "online" and "in-store" profits, we set 'Country' to be the new index.

Is there a different column that we could use as a new index (apart from 'Country' and the original index)?

## `df.pivot`

Yes! We could pivot on the `Type`:


In [ ]:
df.pivot(index='Type', columns='Country')

Yes, there is another column: we could use the 'Type' as the index to pivot to. This sets the index to be each of the unique values of 'Type', which are 'online' and 'in-store'. Each of these needs to be split across each country, so we also get one row per country (GB and US). 


## `df.pivot_table`
* `pivot_table` **only numerical data** and we can add an aggregator:
    * `index`: the columns that will become the new index
    * `columns`: the subcategories to split by
    * `values`: [optional] subset of columns to keep
    * `aggfunc`: [optional] aggregation function for duplicate rows

`.pivot` may not always be suitable because we aren't allowed to have duplicate rows for each subcategory. 

For example, we might have online and in-store results for each quarter and we'd like the pivot table values to be the total for the year. We need to aggregate or sum over each quarter. 

For this purpose we can use `pivot_table` instead of `pivot`. We still pass in an index, columns, and optionally a specific subset of values, but we can also specify an aggregation function (for example, sum). 

Note that `pivot_table` only works with numerical data, where `pivot` can use either numerical or categorical data. 

## Example `df.pivot_table`:

In [ ]:
df = pd.read_csv('data/profits_with_duplicates.csv')
df

Consider this example, which is similar to the previous example but now includes duplicates of the subcategory "Type".

`pivot` would throw an exception if we passed this table, because it does not have an aggregation function. 

## Example `df.pivot_table`:

* `df.pivot` fails because it can't aggregate

In [ ]:
df = df.pivot_table(index='Country', columns='Type', aggfunc='sum')
df

In this case we can instead use `pivot_table` and pass in the optional argument 'sum' as the aggfunc. The default functionality is to take the mean, but we want to sum here so we specify 'sum' as the aggfunc. 

You can see that the `pivot_table` operation has set 'Country' as the new axis just as before, so there are no duplicates of each country in the column. We have the same new columns as when we used `pivot`, except this time the duplicate subcategories have been summed (i.e. in-store sales for each country are now summed into one entry). 


## `melt`
* Reverses the `pivot` operation
* Reshape so we have a column of **categories** (e.g. country) which has **one value** (e.g. profit) **per subcategory**) 
* Arguments:
    * `id_vars`: the index column to be stretched out
    * `value_vars`: the new subcategory (could be multiple of these)

`melt` is the inverse of the `pivot` operation. Where before we were moving the category (for us, country) into the index position, and creating columns for each subcategory, we are now reshaping so that we have a column of the category with one row per subcategory entry. In this case our subcategories are 'in-store' and 'online' so this means that per country we would have two rows for the country `GB` and two rows for the country `US`. 

`melt` takes two key arguments: `id_vars` which is the index column corresponding to the broad category, and `value_vars` which is the new subcategory. We could choose to pass multiple subcategories but for now we'll use one for clarity. 

## Example `melt`:

In [ ]:
df = pd.read_csv('data/profits.csv')
df = df.pivot(index='Country', columns='Type')

# MultiIndex processing - this will be covered in the next section
df = df['Profit'].reset_index() 

df

In [ ]:
pd.melt(df, id_vars='Country', var_name='Type', value_name='Profit')

Consider the example we had earlier where we used `pivot` to set country as the index and have one column for in-store and one column for online sales. 

We have to do a little pre-processing because of Multi-Indexing, which is something you can skip for now and we'll cover in the next section.

We use `melt` to reshape the DataFrame so that we have one row for each of the `value_vars` we specified, which are subcategories to the `id_vars` we gave.

You can see that this reverts us back to the DataFrame we had originally. You can think of the original shape as being 'long' and the pivoted shape as being 'wide' - different scenarios will dictate which one we prefer, but pandas thankfully makes it easy to move between them.

## MultiIndex
* Hierarchical index over several **columns** or **rows**
    * `pivot` (and some other commands) create a multi-index by default


In [ ]:
df = pd.read_csv('data/profits.csv')
df = df.pivot(index='Country', columns='Type')
df

'Profit' is a multi-index with subcolumns.

We have already mentioned that pre-processing of a pivot table can be done with `melt`. This is because `pivot`, by default, creates a MultiIndex. 

A MultiIndex is a way to encapsulate several columns or rows in a group. In our example, in-store and online are subcategories of 'Profit' - they now have a built-in abstraction for being different types of profit. This can be a natural way to express hierarchies of attributes instead of having them in a flat table. 

## Column MultiIndex

Imagine our data is set up in lists:

In [ ]:
# [online, in-store]
gb = [30, 40]
us = [100, 25]

df = pd.DataFrame([gb,us], columns=['online','in-store'])
df

We could combine online and in-store into a MultiIndex called "profit".

A MultiIndex can be used to group together either columns or rows. 

Let's start with columns. Imagine this example where we have the DataFrame set up with two columns, and we would like to group the two columns together into a 'Profit' MultiIndex. 

## Column MultiIndex

Create in three ways: `from_tuples`,`from_arrays`,`from_frames`

In [ ]:
# single index:
df = pd.DataFrame([gb,us], columns=['online','in-store'])

# MultiIndex:
columns = pd.MultiIndex.from_tuples([('profit','online'), ('profit','in-store')])
df = pd.DataFrame([gb,us], columns=columns)
df

`pandas` offers three methods for creating a MultiIndex: from tuples, arrays, set products, or dataframes; all of which work very similarly.

Previously we created a single index by passing in a flat list of columns. To create a MultiIndex, we can pass in a  list of tuples that express the hierarchical mapping from the category to the subcategories. In this case the category is 'profit' and the subcategories are 'online' and 'in-store'. 

Once we have the MultiIndex object, we can create the dataframe as we did before, this time passing the MultiIndex in as the columns (where before we were just passing a flat list). 

## Column Multi-Index

We can chain index or use `loc`:

In [ ]:
df['profit']['online']

In [ ]:
df.loc[:, ('profit', 'online')]

We can continue to index the DataFrame as before, either by chaining together indices in the order category followed by subcategory, or by using `loc`. 

In this example we use the colon to denote that we would like all of the rows, and we use a tuple to specify which category and subcategory we would like to access. If we indexed an individual row then we would get only the online profit for that row.

## Row MultiIndex

In [ ]:
london    = [100,20]
cambridge = [200,30]
new_york  = [300,40]

index = pd.MultiIndex.from_tuples([('GB','London'),('GB','Cambridge'),('US','New York')])

df = pd.DataFrame([london, cambridge, new_york], columns=['profit', 'revenue'], index=index)

df

Very similarly we can create a MultiIndex for rows - in this example we are grouping together cities by their country. 

The syntax is very similar, except this time instead of passing in the MultiIndex to the columns keyword we are passing it in as an Index. This makes sense for a row MultiIndex because the Index is what we use to access each of the rows. 


## Row MultiIndex

We select row subsets in a similar way:

In [ ]:
df.loc["GB", :]

In [ ]:
df.loc[("GB", "London"),:]

The row MultiIndex can be accessed with the usual pandas `loc` syntax. We can access columns using square brackets,
but we need `loc` in order to access via the index of the DataFrame. 

Here are two examples. In the first example, we ask for all columns with "GB" as the category using the colon in the column index. We index via [rows, columns] so a colon after the comma indicates we want all of the columns (profit and revenue).

In the second example we pass a tuple in order to navigate the hierarchy and specify that we want the country and city "GB" and "London", and then we ask for all the columns.


## Stack and Unstack

* Stack: converts inner **column** MultiIndex to a **row** MultiIndex
* Unstack: converts inner **row** MultiIndex to a **column** MultiIndex

`stack` and `unstack` are pandas methods for transposing a MultiIndex, and they interact a little like `pivot` and `melt`. 

We use `stack` when we have a column MultiIndex and we'd like a row MultiIndex. When we want to do the opposite we use `unstack`. Note that the hierarchy is preserved, it's just transposed: we are not flattening any indices here.

## Stack

Consider the **column** MultiIndex we created earlier:

In [ ]:
gb = [30, 40]
us = [100, 25]

columns = pd.MultiIndex.from_tuples([('profit','online'), ('profit','in-store')])
df = pd.DataFrame([gb,us], columns=columns, index=['GB','US'])
df

`stack` flattens "profit" into a single column, multi-indexed by rows "online" and "in-store".

Consider the example we had earlier where we created a column multiindex for different types of profit.

In this case we might want to have a single column for profit and instead have a row MultiIndex for each of the types of profit, so we could use `stack` to transpose the dataframe.

## Stack 

In [ ]:
df

In [ ]:
df.stack()

This might not seem particularly natural in this case, but hopefully it is a good illustration of exactly what to expect from `stack`. 

When we apply `stack`, profit becomes one column, and each country is subcategorised by a row for each type of profit we have. 

## Unstack

Unstack does the opposite, converting the row MultiIndex back to a column MultiIndex:

In [ ]:
df = df.stack()
df.unstack()

By contrast, if we had the row MultiIndex and we instead wanted a column MultiIndex we could use `unstack`, which works just as we would expect, reverting the dataframe back to its previous state. 


## Crosstab

Use `crosstab` to compare aggregations of different values.

In [ ]:
df = pd.read_csv('data/profits_with_duplicates.csv')
df

e.g. `crosstab` could `sum` the profits for each country.

Consider again case where we have duplicate rows for each of the subcategories and we haven't created a pivot or a MultiIndex yet. 

`crosstab` is used to tabulate different columns in the DataFrame. Often its most common usage is to count the number of occurences of each subcategory (e.g. recording the number of entries we have for each country/profit type). 

Another usage might be to use it to sum the total profits, as an alternative to using `pivot_table`. 

## `crosstab` example

In [ ]:
pd.crosstab(df['Country'], columns=df['Type'], values=df['Profit'], aggfunc='sum')

The syntax is very similar to `pivot_table` and in the case the result is also very similar. We specify an `index`, in this case 'Country', and the column that contains the subcategories, in this case 'Type'. 

We also have to specify the values which we want to aggregate, and the aggfunc we want to use (in this case, sum). 

`crosstab`, like `pivot_table`, will return a MultiIndex which you can see here. 

You might wonder what the difference is between the two. 

In general, `crosstab` is very useful for making simple analyses when there isn't a need to filter down the result, and we want things like frequency tables or small tabulations. `pivot_table` allows us to express more complicated queries by letting us use grouping over the index and columns.

## `cut`

Transform **numerical** data into **categorical** data
* group numbers into histogram bins
* each value is replaced by the number it bins to 

In [ ]:
a = np.linspace(0, 9, 10)
a

* e.g. imagine we want to split these numbers up into two bins (<=5 and >5).

`cut` is another useful pandas function, which we can use to quantise numerical data into categorical groups. A simple example is that we might want to compress a list of numbers from 0 to 9 into two groups. 

`cut` automatically splits these into two bins, and returns a list with the invidual values replaced by their bin.

## `cut`

In [ ]:
binned = pd.cut(a, bins=2)
binned

In [ ]:
binned.categories

This might look a bit strange at first. `pandas` has replaced every value in the list with the range it fits into (either 0 to 4.5 or 4.5 to 9). It also has a `categories` attribute which tells us what the two intervals are.

The range opens with a curved bracket and ends with a square bracket, with the square bracket meaning that that number is inclusive. So in the second bin 4.5 is exclusive, and 9.0 is inclusive. This means that the number 4.5 would be grouped into the first bin.

We can now use this numerical data as categorical data instead, or use it to easily create histograms.

## `factorize`

* `factorize` splits data into a dataset `codes, uniques`, assigning a code to each unique value

In [ ]:
codes, uniques = pd.factorize(['GB','GB','US','FR','GB','FR','FR'])
print(codes)
print(uniques)

In [ ]:
uniques[codes[0]]

`factorize` works by assigning a code to each unique value in a dataset. In this case we have a list of countries with duplicates. 

We can `factorize` the list into a pair of codes and unique values, with the uniques representing each of the unique country labels. The values are then replaced by their code, which we can use to index uniques to retrieve the original value. 

This might be useful for example if we had a very large dataset with lots of duplicates: it may be cheaper to store duplicates of the integer codes than of long strings.  However, its main use is in scenarios where we only care about whether or not entries are distinct (and not their actual value). 